<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

from openai import OpenAI

import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 944fe12b93e5c9f60a6dc6b0d18688e891a1249f

pandas    : 2.1.4
json      : 2.0.9
termcolor : 2.4.0
matplotlib: 3.8.0
tqdm      : 4.66.2
numpy     : 1.26.4
watermark : 2.4.3



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

Then we are ready to instantiate the model

In [4]:
client = OpenAI()

We start by getting a list of supported models.

In [5]:
model_list = json.loads(client.models.list().json())["data"]

In total we have 32 models

In [6]:
len(model_list)

33

Along with some information about each model...

In [7]:
model_list[:3]

[{'id': 'dall-e-3',
  'created': 1698785189,
  'object': 'model',
  'owned_by': 'system'},
 {'id': 'whisper-1',
  'created': 1677532384,
  'object': 'model',
  'owned_by': 'openai-internal'},
 {'id': 'davinci-002',
  'created': 1692634301,
  'object': 'model',
  'owned_by': 'system'}]

But let's just get a list of model names

In [8]:
print("\n".join(sorted([model["id"] for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-1106-vision-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4-vision-preview
gpt-4o
gpt-4o-2024-05-13
gpt-4o-test-shared
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [9]:
%%time
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 6.05 ms, sys: 1.79 ms, total: 7.84 ms
Wall time: 873 ms


Which produces a response object

In [10]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a named tuple

In [11]:
pprint(response.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's weakness is kryptonite, a mineral from his home planet of Krypton that weakens and can even kill him.", role='assistant', function_call=None, tool_calls=None))]


The model answer can be found in the "message" dictionary inside the "choices" list

In [12]:
response.choices[0].message.content

"Superman's weakness is kryptonite, a mineral from his home planet of Krypton that weakens and can even kill him."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [13]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 5.78 ms, sys: 1.87 ms, total: 7.64 ms
Wall time: 3.78 s


And we can access each of the answers individually int he choices list

In [14]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
There are several different types of Kryptonite, each with unique effects on Superman depending on its color:

1. Green Kryptonite: This is the most common form of Kryptonite and is deadly to Superman. Exposure to green Kryptonite weakens him, making him lose his powers and eventually leading to his death if he is not removed from its presence.

2. Red Kryptonite: Red Kryptonite has unpredictable effects on Superman, with each exposure causing different temporary changes or distortions in his abilities or behavior.

3. Blue Kryptonite: Blue Kryptonite is specifically harmful to Bizarro, a flawed clone of Superman. It has no effect on the real Superman.

4. Gold Kryptonite: Gold Kryptonite permanently removes Superman's powers and abilities, essentially turning him into a normal human being.

5. White Kryptonite: White Kryptonite is lethal to all plant life, causing it to wither and die. It has no effect on Superman.

6. Black Kryptonite: Black Kryptonite has the ability to sp

In [15]:
response.usage

CompletionUsage(completion_tokens=607, prompt_tokens=17, total_tokens=624)

# Temperature

In [16]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.5
)

CPU times: user 5.68 ms, sys: 1.4 ms, total: 7.08 ms
Wall time: 3.95 s


In [17]:
print(response.choices[0].message.content)

Once there was a school that held a talent show every year. This year, Lily had decided to participate. Despite being a bit nervous, she chose to sing because it was something that had always brought her joy.

However, as the day of the talent show approached, Lily's nerves got the best of her and she began to doubt herself. On the day of the talent show, she felt sick to her stomach and barely ate breakfast.

But when it was her turn to perform, something incredible happened. As soon as she started singing, all her doubts and fears disappeared. She put her heart and soul into the performance and sang with such emotion that the entire audience was captivated.

When she finished, there was a moment of silence before the whole room erupted in applause. Lily couldn't believe it. The headteacher even stood up and declared her performance the best of the night.

From that moment on, Lily gained confidence in her talent and realized that as long as she stayed true to herself, she could achie

In [18]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 8.19 ms, sys: 2.57 ms, total: 10.8 ms
Wall time: 3.98 s


In [19]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the mountains, there lived a young girl named Lily. She was known for her kindness and generosity, always willing to help those in need.

One day, a terrible storm hit the village, causing widespread damage and leaving many families homeless. Lily knew she had to do something to help. She gathered her friends and together they started a fundraiser to rebuild the homes that were destroyed.

Through hard work and determination, they were able to raise enough money to not only rebuild the homes, but also provide food and clothing for those affected by the storm. The village was filled with gratitude for Lily and her friends' selfless actions.

From that day on, Lily became known as the hero of the village, a shining example of how one person's kindness and compassion can make a difference in the lives of others. And she continued to spread love and positivity wherever she went, inspiring those around her to do the same.


# Function Calls

In [20]:
def chat(messages, functions):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        # Define the functions the model is allowed to use
        functions=functions
    )
    
    return response

In [21]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        print(message)
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and message['function_call']:
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and not message['function_call']:
            print(colored(f"assistant: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "function":
            print(colored(f"function ({message.name}): {message.content}\n", role_to_color[message.role]))


Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [22]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [23]:
messages = []

messages.append(
    {"role": "system", 
     "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    })

messages.append(
    {"role": "user", 
     "content": "What's the weather like today"
    })

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [24]:
messages.append(
    {"role": "user", 
     "content": "I'm in New York, NY."
    })

In [25]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response.choices[0].message
messages.append({
 "role":  assistant_message.role,
 "content":  assistant_message.content,
 "function_call":  assistant_message.function_call,
})

In [26]:
pretty_print_conversation(messages)

{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

{'role': 'user', 'content': "What's the weather like today"}
user: What's the weather like today

{'role': 'user', 'content': "I'm in New York, NY."}
user: I'm in New York, NY.

{'role': 'assistant', 'content': None, 'function_call': FunctionCall(arguments='{"location":"New York, NY","format":"celsius"}', name='get_current_weather')}
assistant: FunctionCall(arguments='{"location":"New York, NY","format":"celsius"}', name='get_current_weather')



## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [27]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

This last-minute change means we can't spend a lot of time on the client's project.


# Formatted output

In [28]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = client.chat.completions.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 9.55 ms, sys: 1.95 ms, total: 11.5 ms
Wall time: 4.48 s


In [29]:
json_output = response.choices[0].message.content

In [30]:
output = json.loads(json_output)

In [31]:
output["ingredients"]

['1 cup all-purpose flour',
 '1 tablespoon sugar',
 '1 teaspoon baking powder',
 '1/2 teaspoon baking soda',
 '1/4 teaspoon salt',
 '1 cup buttermilk',
 '1 large egg',
 '2 tablespoons melted butter',
 '1 cup fresh blueberries',
 'Butter or oil for cooking']

In [32]:
output["instructions"]

['In a large bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.',
 'In a separate bowl, whisk together the buttermilk, egg, and melted butter.',
 'Pour the wet ingredients into the dry ingredients and stir until just combined. The batter may be slightly lumpy.',
 'Gently fold in the fresh blueberries.',
 'Heat a non-stick skillet or griddle over medium heat and add a little butter or oil.',
 'Pour 1/4 cup of batter for each pancake onto the skillet. Cook until bubbles form on the surface of the pancakes and the edges look set, about 2-3 minutes.',
 'Flip the pancakes and cook for another 1-2 minutes, or until golden brown.',
 'Repeat with the remaining batter, adding more butter or oil to the skillet as needed.',
 'Serve the blueberry pancakes warm with maple syrup or your favorite pancake toppings.']

# Translation

In [33]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [34]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [35]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [36]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [37]:
print(response.choices[0].message.content)

Fruit,Color,Flavor
neoskizzles,Purple,Candy
loheckles,Grayish blue,Tart
pounits,Bright green,Savory
loopnovas,Neon pink,Cotton candy
glowls,Pale orange,Sour and bitter


In [38]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [39]:
print(response.choices[0].message.content)

Pounits are bright green fruits that are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>